In [1]:
import apache_log_parser
import glob
import os
import datetime
from pprint import pprint

In [6]:
def file_read():
    #ファイルの読み込み
    files = {}
    test_log = {}
    test_logs = []
    i = 0
    for file in glob.glob("test_data*.txt"):
        i+=1
        files[i] = file
        with open(file, "r") as f:
            for test_log in f:
                yield test_log


In [3]:
#指定した期間の入力
def time_entry():
    while True:
        try:
            print("year_start")
            y_s = int(input())

            print("month_start")
            m_s = int(input())
            if(m_s > 12): 
                print("12より下の数字を入力してください")
                continue

            print("day_start")
            d_s = int(input())
            if(d_s > 31):
                print("31より下の数字を入力してください")
                continue

            print("year_finish")
            y_f = int(input())

            print("month_finish")
            m_f = int(input())
            if(m_f > 12): 
                print("12より下の数字を入力してください")
                continue

            print("day_finish")
            d_f = int(input())
            if(d_f > 31):
                print("31より下の数字を入力してください")
                continue
            
            if(y_s < y_f):
                if(m_s < m_f):
                    if(d_s < d_f):
                        print('期間設定がおかしいです')
                        continue
            break
        except ValueError:
            print("数値を代入してください")

    start = datetime.date(y_s,m_s,d_s)
    finish = datetime.date(y_f,m_f,d_f)
    print(start)
    print(finish)
    return start,finish

In [4]:
#ログのデータの読み込み
def logs_read(test_log,start,finish):
    line_parser = apache_log_parser.make_parser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")
    log = [] #ログのデータ
    
    log_line_data = line_parser(test_log)
    log_time = log_line_data["time_received_datetimeobj"]
    log_time_date = datetime.date(log_time.year,log_time.month,log_time.day)
    if(start <= log_time_date <= finish):
        log = log_line_data
    return log

In [5]:
if __name__ == '__main__':
    start,finish = time_entry()
    host_name = {} #ホスト名ごとのアクセス件数
    log_hour = {} #各時間帯のアクセス件数のデータ
    for i in range (0,24):
        log_hour[i] = 0
        
    for test_log in file_read():
        log = logs_read(test_log,start,finish)
        #ログの時間帯による集計
        hour = log["time_received_datetimeobj"].hour
        log_hour[hour] += 1
        #ホスト名ごとによる集計
        host_name.setdefault(log["remote_host"],0) 
        host_name[log["remote_host"]] +=1
        
    pprint(log_hour)
    print(host_name)

year_start
2005
month_start
1
day_start
1
year_finish
2005
month_finish
12
day_finish
21
2005-01-01
2005-12-21
{0: 59403,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0}
{'10.2.3.4': 59403}
